In [1]:
# 필요한 라이브러리 임포트
import os
import time
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm
import re

print("라이브러리 임포트 완료")


라이브러리 임포트 완료


In [8]:
# API 키 로드 및 데이터 파일 로드
load_dotenv()
SERVICE_KEY = os.getenv("BUILDING_HUB_API_KEY")
if not SERVICE_KEY:
    raise ValueError("환경변수 BUILDING_HUB_API_KEY가 설정되어 있지 않습니다.")

print("API 키 로드 완료")

# API요청 시트에서 데이터 로드
def load_api_request_data():
    """batch_stage_matching_result_final_추가작업 요청.xlsx 파일의 API요청 시트에서 데이터 로드"""
    
    file_path = "../data/batch_stage_matching_result_final_추가작업 요청.xlsx"
    
    try:
        # API요청 시트 로드
        df = pd.read_excel(file_path, sheet_name="API 수집")
        
        print(f"✓ API요청 시트 로드 성공!")
        print(f"  데이터 크기: {df.shape}")
        print(f"  컬럼: {list(df.columns)}")
        
        # new_주소 컬럼 확인
        if 'new_주소' in df.columns:
            print(f"  new_주소 컬럼 발견!")
            print(f"  new_주소 총 개수: {len(df)}건")
            print(f"  new_주소 결측치: {df['new_주소'].isna().sum()}개")
            print(f"  new_주소 유효한 데이터: {df['new_주소'].notna().sum()}개")
            
            # 샘플 확인
            print(f"\nnew_주소 샘플:")
            sample_addresses = df['new_주소'].dropna().head(5)
            for i, addr in enumerate(sample_addresses, 1):
                print(f"  {i}. {addr}")
            
            return df
        else:
            print(f"  ❌ new_주소 컬럼을 찾을 수 없습니다.")
            print(f"  사용 가능한 컬럼: {list(df.columns)}")
            return None
            
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 파일 로드 오류: {e}")
        return None

# 데이터 로드 실행
df = load_api_request_data()


API 키 로드 완료
✓ API요청 시트 로드 성공!
  데이터 크기: (148, 1)
  컬럼: ['new_주소']
  new_주소 컬럼 발견!
  new_주소 총 개수: 148건
  new_주소 결측치: 0개
  new_주소 유효한 데이터: 148개

new_주소 샘플:
  1. 강원특별자치도 정선군 북평면 남평리 66-3, 한국가스공사생활관가동,경비실
  2. 강원특별자치도 태백시 황지동 49-225
  3. 강원특별자치도 평창군 용평면 백옥포리 251
  4. 강원특별자치도 평창군 용평면 백옥포리 251
  5. 경기도 고양시 덕양구 지축동 737-1 지축차량기지


In [9]:
# 기존 건축HUB 데이터에서 주소-코드 매핑 테이블 생성
def create_address_code_mapping():
    """기존 수집된 건축HUB 데이터에서 주소 → 코드 매핑 테이블 생성"""
    
    # 기존 수집된 파일들 확인 (result 폴더 내부)
    csv_files = [
        "../result/03표제부_건축HUB.csv",
        "../result/기본개요_건축HUB.csv"
    ]
    
    excel_files = [
        "../result/df_basic.xlsx",
        "../result/df_title.xlsx"
    ]
    
    mapping_df = None
    
    # CSV 파일 처리
    for file_path in csv_files:
        if os.path.exists(file_path):
            print(f"매핑 데이터 로드: {file_path}")
            try:
                temp_df = pd.read_csv(file_path, encoding='utf-8-sig')
                
                # 필요한 컬럼이 있는지 확인
                required_cols = ['platPlc', 'sigunguCd', 'bjdongCd']
                if all(col in temp_df.columns for col in required_cols):
                    # 매핑에 필요한 컬럼만 추출 (결측치 제거)
                    temp_mapping = temp_df[required_cols].dropna().drop_duplicates()
                    
                    if mapping_df is None:
                        mapping_df = temp_mapping
                    else:
                        mapping_df = pd.concat([mapping_df, temp_mapping]).drop_duplicates()
                        
                    print(f"  → {len(temp_mapping)}개 주소-코드 매핑 추가")
                else:
                    print(f"  → 필요한 컬럼 없음: {temp_df.columns.tolist()}")
                    
            except Exception as e:
                print(f"  → 파일 로드 오류: {e}")
        else:
            print(f"파일 없음: {file_path}")
    
    # Excel 파일 처리
    for file_path in excel_files:
        if os.path.exists(file_path):
            print(f"매핑 데이터 로드: {file_path}")
            try:
                temp_df = pd.read_excel(file_path)
                
                # 필요한 컬럼이 있는지 확인
                required_cols = ['platPlc', 'sigunguCd', 'bjdongCd']
                if all(col in temp_df.columns for col in required_cols):
                    # 매핑에 필요한 컬럼만 추출 (결측치 제거)
                    temp_mapping = temp_df[required_cols].dropna().drop_duplicates()
                    
                    if mapping_df is None:
                        mapping_df = temp_mapping
                    else:
                        mapping_df = pd.concat([mapping_df, temp_mapping]).drop_duplicates()
                        
                    print(f"  → {len(temp_mapping)}개 주소-코드 매핑 추가")
                else:
                    print(f"  → 필요한 컬럼 없음: {temp_df.columns.tolist()}")
                    
            except Exception as e:
                print(f"  → 파일 로드 오류: {e}")
        else:
            print(f"파일 없음: {file_path}")
    
    if mapping_df is not None:
        # 중복 제거
        mapping_df = mapping_df.drop_duplicates().reset_index(drop=True)
        print(f"\n총 {len(mapping_df)}개의 고유 주소-코드 매핑 생성됨")
        return mapping_df
    else:
        print("주소-코드 매핑 데이터를 찾을 수 없습니다.")
        return None

# 매핑 테이블 생성
print("=== 주소-코드 매핑 테이블 생성 ===")
address_mapping = create_address_code_mapping()

if address_mapping is not None:
    print("\n=== 주소-코드 매핑 샘플 ===")
    print(address_mapping.head())
    
    # 시군구별 분포 확인
    print(f"\n시군구코드 분포 (상위 10개):")
    print(address_mapping['sigunguCd'].value_counts().head(10))
else:
    print("⚠️ 매핑 데이터가 없어서 직접 파싱만 가능합니다.")


=== 주소-코드 매핑 테이블 생성 ===
매핑 데이터 로드: ../result/03표제부_건축HUB.csv


C:\Users\HyeonPoong Lee\AppData\Local\Temp\ipykernel_27640\94843676.py:23: DtypeWarning: Columns (20,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_path, encoding='utf-8-sig')


  → 1809개 주소-코드 매핑 추가
매핑 데이터 로드: ../result/기본개요_건축HUB.csv


C:\Users\HyeonPoong Lee\AppData\Local\Temp\ipykernel_27640\94843676.py:23: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_path, encoding='utf-8-sig')


  → 2014개 주소-코드 매핑 추가
매핑 데이터 로드: ../result/df_basic.xlsx
  → 필요한 컬럼 없음: ['순번', '대지위치', '시군구코드', '법정동코드', '대지구분코드', '번', '지', '관리건축물대장PK', '관리상위건축물대장PK', '건물_아이디', '대장구분코드', '대장구분코드명', '대장종류코드', '대장종류코드명', '도로명대지위치', '건물명', '특수지명', '블록', '로트', '외필지수', '새주소도로코드', '새주소법정동코드', '새주소지상지하코드', '새주소본번', '새주소부번', '지역코드', '지구코드', '구역코드', '지역코드명', '지구코드명', '구역코드명', '생성일자', 'MGM_BLD_PK']
매핑 데이터 로드: ../result/df_title.xlsx
  → 필요한 컬럼 없음: ['순번', '대지위치', '시군구코드', '법정동코드', '대지구분코드', '번', '지', '관리건축물대장PK', '대장구분코드', '대장구분코드명', '대장종류코드', '대장종류코드명', '도로명대지위치', '건물명', '특수지명', '블록', '로트', '외필지수', '새주소도로코드', '새주소법정동코드', '새주소지상지하코드', '새주소본번', '새주소부번', '동명칭', '주부속구분코드', '주부속구분코드명', '대지면적(㎡)', '건축면적(㎡)', '건폐율(%)', '연면적(㎡)', '용적률산정연면적(㎡)', '용적률(%)', '구조코드', '구조코드명', '기타구조', '주용도코드', '주용도코드명', '기타용도', '지붕코드', '지붕코드명', '기타지붕', '세대수(세대)', '가구수(가구)', '높이(m)', '지상층수', '지하층수', '승용승강기수', '비상용승강기수', '부속건축물수', '부속건축물면적(㎡)', '총동연면적(㎡)', '옥내기계식대수(대)', '옥내기계식면적(㎡)', '옥외기계식대수(대)', '옥외기계식면적(㎡)', '옥내자주식대수(대)', '옥내자주식면적(㎡)', '옥

In [10]:
# 스마트한 주소 매칭 함수
def smart_address_matching(target_address, mapping_df):
    """
    유사도 기반 주소 매칭
    
    Parameters:
    -----------
    target_address : str
        매칭할 대상 주소
    mapping_df : DataFrame
        기존 주소-코드 매핑 테이블
        
    Returns:
    --------
    dict : 매칭된 시군구코드, 법정동코드
    """
    if mapping_df is None or len(mapping_df) == 0:
        return None
    
    # 대상 주소 정규화
    target_clean = re.sub(r'[^\w\s]', '', target_address)
    target_clean = re.sub(r'\s+', ' ', target_clean).strip()
    
    best_match = None
    best_score = 0
    
    # 매핑 테이블 순회하며 매칭 시도
    for idx, row in mapping_df.iterrows():
        map_address = str(row['platPlc'])
        map_clean = re.sub(r'[^\w\s]', '', map_address)
        map_clean = re.sub(r'\s+', ' ', map_clean).strip()
        
        # 1. 완전 일치
        if target_clean == map_clean:
            return {
                'sigunguCd': row['sigunguCd'],
                'bjdongCd': row['bjdongCd'],
                'matched_address': map_address,
                'match_type': '완전일치'
            }
        
        # 2. 포함 관계 매칭
        if target_clean in map_clean or map_clean in target_clean:
            # 길이 유사도 계산
            score = min(len(target_clean), len(map_clean)) / max(len(target_clean), len(map_clean))
            if score > best_score:
                best_score = score
                best_match = {
                    'sigunguCd': row['sigunguCd'],
                    'bjdongCd': row['bjdongCd'],
                    'matched_address': map_address,
                    'match_type': f'포함매칭({score:.2f})'
                }
    
    # 3. 토큰 기반 매칭 (앞에서 매칭되지 않은 경우)
    if best_match is None:
        target_tokens = set(target_clean.split())
        
        for idx, row in mapping_df.iterrows():
            map_address = str(row['platPlc'])
            map_clean = re.sub(r'[^\w\s]', '', map_address)
            map_tokens = set(map_clean.split())
            
            # 공통 토큰 비율
            if len(target_tokens) > 0 and len(map_tokens) > 0:
                common_tokens = target_tokens.intersection(map_tokens)
                score = len(common_tokens) / max(len(target_tokens), len(map_tokens))
                
                if score > 0.5 and score > best_score:  # 50% 이상 매칭
                    best_score = score
                    best_match = {
                        'sigunguCd': row['sigunguCd'],
                        'bjdongCd': row['bjdongCd'],
                        'matched_address': map_address,
                        'match_type': f'토큰매칭({score:.2f})'
                    }
    
    return best_match

# 매칭 테스트
if address_mapping is not None and df is not None:
    print("\n=== 스마트 주소 매칭 테스트 ===")
    
    # new_주소가 있는 데이터 샘플
    test_addresses = df[df['new_주소'].notna()]['new_주소'].head(5)
    
    for i, addr in enumerate(test_addresses, 1):
        print(f"\n{i}. 대상 주소: {addr}")
        
        # 스마트 매칭 시도
        match_result = smart_address_matching(addr, address_mapping)
        
        if match_result:
            print(f"   ✓ 매칭 성공!")
            print(f"     시군구코드: {match_result['sigunguCd']}")
            print(f"     법정동코드: {match_result['bjdongCd']}")
            print(f"     매칭방식: {match_result['match_type']}")
            print(f"     매칭된 주소: {match_result['matched_address']}")
        else:
            print(f"   ✗ 매칭 실패")

else:
    print("주소 매핑 데이터 또는 대상 데이터가 없습니다.")



=== 스마트 주소 매칭 테스트 ===

1. 대상 주소: 강원특별자치도 정선군 북평면 남평리 66-3, 한국가스공사생활관가동,경비실
   ✗ 매칭 실패

2. 대상 주소: 강원특별자치도 태백시 황지동 49-225
   ✓ 매칭 성공!
     시군구코드: 51190
     법정동코드: 10100
     매칭방식: 토큰매칭(0.75)
     매칭된 주소: 강원특별자치도 태백시 황지동 20-8번지

3. 대상 주소: 강원특별자치도 평창군 용평면 백옥포리 251
   ✗ 매칭 실패

4. 대상 주소: 강원특별자치도 평창군 용평면 백옥포리 251
   ✗ 매칭 실패

5. 대상 주소: 경기도 고양시 덕양구 지축동 737-1 지축차량기지
   ✗ 매칭 실패


In [11]:
# new_주소를 API 파라미터로 변환하는 함수
def parse_address_to_api_params(address, mapping_df=None):
    """
    new_주소를 API 요청에 필요한 파라미터로 변환
    
    Parameters:
    -----------
    address : str
        예: "강원특별자치도 정선군 북평면 남평리 66-3"
    mapping_df : DataFrame
        주소-코드 매핑 테이블
    
    Returns:
    --------
    dict : API 파라미터
    """
    if pd.isna(address) or not isinstance(address, str):
        return None
    
    # 기본값 설정
    result = {
        'SIGUNGU_CD': '',
        'BJDONG_CD': '',
        'PLAT_GB_CD': '0',  # 기본값: 일반(0)
        'BUN': '',
        'JI': '',
        'original_address': address,
        'match_method': '직접파싱'
    }
    
    try:
        # 1. 스마트 매칭 시도 (매핑 데이터가 있는 경우)
        if mapping_df is not None:
            match_result = smart_address_matching(address, mapping_df)
            if match_result:
                result['SIGUNGU_CD'] = match_result['sigunguCd']
                result['BJDONG_CD'] = match_result['bjdongCd']
                result['match_method'] = f"스마트매칭({match_result['match_type']})"
                result['matched_address'] = match_result['matched_address']
        
        # 2. 번지 추출 (다양한 패턴 지원)
        # 패턴 1: "66-3번지", "100번지"
        bun_ji_pattern = r'(\d+)(?:-(\d+))?번지'
        bun_ji_match = re.search(bun_ji_pattern, address)
        
        if bun_ji_match:
            bun = bun_ji_match.group(1)
            ji = bun_ji_match.group(2) if bun_ji_match.group(2) else '0'
            result['BUN'] = bun
            result['JI'] = ji
        else:
            # 패턴 2: "66-3", "100" (번지 없이)
            number_pattern = r'(\d+)(?:-(\d+))?(?:\s|$)'
            number_match = re.search(number_pattern, address)
            if number_match:
                bun = number_match.group(1)
                ji = number_match.group(2) if number_match.group(2) else '0'
                result['BUN'] = bun
                result['JI'] = ji
        
        # 3. '산' 여부 확인 (대지구분코드)
        if '산' in address:
            result['PLAT_GB_CD'] = '1'  # 산: 1
        
        # 4. 매핑 실패한 경우 상태 표시
        if not result['SIGUNGU_CD'] and not result['BJDONG_CD']:
            result['match_method'] = '매핑실패'
        
    except Exception as e:
        print(f"주소 파싱 오류: {address} - {e}")
        return None
    
    return result

# 데이터 처리 및 API 파라미터 생성
if df is not None:
    print("\n=== new_주소 → API 파라미터 변환 ===")
    
    # new_주소가 있는 데이터만 필터링
    df_with_address = df[df['new_주소'].notna()].copy()
    print(f"new_주소가 있는 데이터: {len(df_with_address)}건")
    
    # 전체 데이터에 대해 파싱 통계 생성
    parsing_stats = {'성공': 0, '실패': 0, '매핑없음': 0}
    api_ready_count = 0
    
    print(f"\n샘플 주소 파싱 테스트 (처음 10개):")
    
    sample_count = min(10, len(df_with_address))
    for i, (idx, row) in enumerate(df_with_address.head(sample_count).iterrows()):
        addr = row['new_주소']
        print(f"\n{i+1}. 원본: {addr}")
        
        parsed = parse_address_to_api_params(addr, address_mapping)
        if parsed:
            print(f"   변환: BUN={parsed['BUN']}, JI={parsed['JI']}, 산={parsed['PLAT_GB_CD']}")
            print(f"   매칭: {parsed['match_method']}")
            
            # API 호출 가능한지 확인
            if parsed['SIGUNGU_CD'] and parsed['BJDONG_CD'] and parsed['BUN']:
                print(f"   상태: ✓ API 호출 가능")
                api_ready_count += 1
                parsing_stats['성공'] += 1
            elif not parsed['SIGUNGU_CD'] or not parsed['BJDONG_CD']:
                print(f"   상태: ⚠️ 시군구/법정동 코드 없음")
                parsing_stats['매핑없음'] += 1
            else:
                print(f"   상태: ❌ 번지 정보 없음")
                parsing_stats['실패'] += 1
        else:
            print(f"   상태: ❌ 파싱 실패")
            parsing_stats['실패'] += 1
    
    print(f"\n=== 샘플 파싱 결과 요약 ===")
    print(f"API 호출 가능: {parsing_stats['성공']}건")
    print(f"매핑 실패: {parsing_stats['매핑없음']}건") 
    print(f"파싱 실패: {parsing_stats['실패']}건")
    
else:
    print("데이터가 로드되지 않았습니다.")



=== new_주소 → API 파라미터 변환 ===
new_주소가 있는 데이터: 148건

샘플 주소 파싱 테스트 (처음 10개):

1. 원본: 강원특별자치도 정선군 북평면 남평리 66-3, 한국가스공사생활관가동,경비실
   변환: BUN=, JI=, 산=0
   매칭: 매핑실패
   상태: ⚠️ 시군구/법정동 코드 없음

2. 원본: 강원특별자치도 태백시 황지동 49-225
   변환: BUN=49, JI=225, 산=0
   매칭: 스마트매칭(토큰매칭(0.75))
   상태: ✓ API 호출 가능

3. 원본: 강원특별자치도 평창군 용평면 백옥포리 251
   변환: BUN=251, JI=0, 산=0
   매칭: 매핑실패
   상태: ⚠️ 시군구/법정동 코드 없음

4. 원본: 강원특별자치도 평창군 용평면 백옥포리 251
   변환: BUN=251, JI=0, 산=0
   매칭: 매핑실패
   상태: ⚠️ 시군구/법정동 코드 없음

5. 원본: 경기도 고양시 덕양구 지축동 737-1 지축차량기지
   변환: BUN=737, JI=1, 산=0
   매칭: 매핑실패
   상태: ⚠️ 시군구/법정동 코드 없음

6. 원본: 경기도 남양주시 이패동 387
   변환: BUN=387, JI=0, 산=0
   매칭: 매핑실패
   상태: ⚠️ 시군구/법정동 코드 없음

7. 원본: 경기도 부천시 소사구 소사본동 51번지, 소사역사
   변환: BUN=51, JI=0, 산=0
   매칭: 스마트매칭(토큰매칭(0.67))
   상태: ✓ API 호출 가능

8. 원본: 경기도 수원시 팔달구 인계동 334-1번지, 수원청소년문화센타 체육시설 ) 권광로 293
   변환: BUN=334, JI=1, 산=0
   매칭: 매핑실패
   상태: ⚠️ 시군구/법정동 코드 없음

9. 원본: 경기도 안산시 상록구 사동 1342-2
   변환: BUN=1342, JI=2, 산=1
   매칭: 스마트매칭(토큰매칭(0.80))
   상태: ✓ API 호출 가능

10. 원본: 경기도 

In [12]:
# 건축HUB API 호출 함수
def fetch_basic_outline_data(params):
    """
    기본개요 데이터를 API로 조회하는 함수
    
    Parameters:
    -----------
    params : dict
        SIGUNGU_CD, BJDONG_CD, PLAT_GB_CD, BUN, JI 정보가 있는 딕셔너리
    
    Returns:
    --------
    list : API 응답 결과 리스트
    """
    api_params = {
        "serviceKey": SERVICE_KEY,
        "sigunguCd": params.get("SIGUNGU_CD", ""),
        "bjdongCd": params.get("BJDONG_CD", ""),
        "platGbCd": params.get("PLAT_GB_CD", "0"),
        "bun": str(params.get("BUN", "")).zfill(4),      # "12" → "0012"
        "ji": str(params.get("JI", "0")).zfill(4),       # "0"  → "0000"
        "numOfRows": "100",
        "pageNo": "1",
        "_type": "json",
    }
    
    try:
        response = requests.get(
            "https://apis.data.go.kr/1613000/BldRgstHubService/getBrBasisOulnInfo",
            params=api_params,
            timeout=30
        )
        response.raise_for_status()
        
        data = response.json()
        
        # 응답 구조 확인
        body = data.get("response", {}).get("body", {})
        items = body.get("items", {})
        
        # items가 None이면 빈 리스트 반환
        if not items:
            return []
        
        # item 추출
        item_data = items.get("item")
        if not item_data:
            return []
        
        # API가 dict 하나만 줄 수도, list를 줄 수도 있으므로 통일
        return item_data if isinstance(item_data, list) else [item_data]
        
    except requests.exceptions.Timeout:
        print(f"API 호출 타임아웃")
        return []
    except requests.exceptions.RequestException as e:
        print(f"API 호출 네트워크 오류: {e}")
        return []
    except Exception as e:
        print(f"API 호출 기타 오류: {e}")
        return []

# 전체 데이터 수집 함수
def collect_basic_outline_data():
    """API요청 시트의 new_주소 데이터로 기본개요 일괄 수집"""
    
    if df is None:
        print("❌ 데이터가 로드되지 않았습니다.")
        return None
    
    # new_주소가 있는 데이터만 필터링
    df_with_address = df[df['new_주소'].notna()].copy()
    
    if len(df_with_address) == 0:
        print("❌ new_주소가 있는 데이터가 없습니다.")
        return None
    
    # 모든 데이터에 대해 API 파라미터 생성
    api_params_list = []
    parsing_stats = {'성공': 0, '실패': 0, '매핑없음': 0}
    
    print(f"\n=== 전체 데이터 주소 파싱 시작 ===")
    print(f"처리할 데이터: {len(df_with_address)}건")
    
    for idx, row in tqdm(df_with_address.iterrows(), total=len(df_with_address), desc="주소 파싱"):
        parsed = parse_address_to_api_params(row['new_주소'], address_mapping)
        
        if parsed:
            parsed['original_index'] = idx
            
            # API 호출 가능한지 확인 (시군구코드, 법정동코드, 번지가 모두 있어야 함)
            if parsed['SIGUNGU_CD'] and parsed['BJDONG_CD'] and parsed['BUN']:
                api_params_list.append(parsed)
                parsing_stats['성공'] += 1
            elif not parsed['SIGUNGU_CD'] or not parsed['BJDONG_CD']:
                parsing_stats['매핑없음'] += 1
            else:
                parsing_stats['실패'] += 1
        else:
            parsing_stats['실패'] += 1
    
    # 파싱 결과 요약
    print(f"\n=== 주소 파싱 결과 ===")
    print(f"✓ API 호출 가능: {parsing_stats['성공']}건")
    print(f"⚠️ 매핑 실패: {parsing_stats['매핑없음']}건")
    print(f"❌ 파싱 실패: {parsing_stats['실패']}건")
    
    if parsing_stats['성공'] == 0:
        print("API 호출할 데이터가 없습니다.")
        return None
    
    # 매칭 방법별 통계
    if api_params_list:
        match_methods = [p['match_method'] for p in api_params_list]
        print(f"\n매칭 방법별 분포:")
        for method, count in pd.Series(match_methods).value_counts().items():
            print(f"  {method}: {count}건")
    
    # 사용자 확인
    proceed = input(f"\n{len(api_params_list)}건의 데이터에 대해 API를 호출하시겠습니까? (y/n): ")
    
    if proceed.lower() != 'y':
        print("API 호출을 취소했습니다.")
        return None
    
    # API 호출 실행
    all_records = []
    failed_count = 0
    
    print(f"\n=== 기본개요 데이터 수집 시작 ===")
    
    for i, params in enumerate(tqdm(api_params_list, desc="API 호출 중")):
        try:
            records = fetch_basic_outline_data(params)
            if records:
                # 원본 정보 추가
                for record in records:
                    record['original_index'] = params['original_index']
                    record['original_address'] = params['original_address']
                    record['match_method'] = params['match_method']
                    if 'matched_address' in params:
                        record['matched_address'] = params['matched_address']
                all_records.extend(records)
            else:
                failed_count += 1
            
            # API 호출 간격 조절 (초당 10회 제한)
            time.sleep(0.1)
            
        except Exception as e:
            print(f"오류 발생 (인덱스 {params['original_index']}): {e}")
            failed_count += 1
    
    # 결과 저장
    if all_records:
        df_result = pd.DataFrame(all_records)
        
        # 현재 시각을 파일명에 포함하고 result 폴더에 저장
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"../result/기본개요_API요청시트_{timestamp}.csv"
        
        # result 폴더가 없으면 생성
        os.makedirs("../result", exist_ok=True)
        
        df_result.to_csv(filename, index=False, encoding="utf-8-sig")
        
        print(f"\n=== 수집 완료 ===")
        print(f"✓ 총 {len(df_result)}건 수집됨")
        print(f"❌ 실패: {failed_count}건")
        print(f"💾 저장 파일: {filename}")
        
        # 결과 미리보기
        print(f"\n=== 결과 미리보기 ===")
        display_cols = ['platPlc', 'original_address', 'match_method']
        available_cols = [col for col in display_cols if col in df_result.columns]
        if available_cols:
            print(df_result[available_cols].head())
        else:
            print(df_result.head())
        
        return df_result
        
    else:
        print("❌ 수집된 데이터가 없습니다.")
        return None

# 함수 준비 완료 안내
print("\n" + "="*60)
print("🚀 API요청 시트 기본개요 데이터 수집 준비 완료!")
print("="*60)
print("실행하려면: collect_basic_outline_data() 를 호출하세요.")
print("\n📋 처리 과정:")
print("1. API요청 시트의 new_주소 데이터 로드")
print("2. 기존 건축HUB 데이터에서 주소-코드 매핑")
print("3. 스마트 주소 매칭 (완전일치 → 포함매칭 → 토큰매칭)")
print("4. 번지 정보 추출 및 API 파라미터 생성")
print("5. 건축HUB API 호출 및 기본개요 데이터 수집")
print("6. 결과를 CSV 파일로 저장")



🚀 API요청 시트 기본개요 데이터 수집 준비 완료!
실행하려면: collect_basic_outline_data() 를 호출하세요.

📋 처리 과정:
1. API요청 시트의 new_주소 데이터 로드
2. 기존 건축HUB 데이터에서 주소-코드 매핑
3. 스마트 주소 매칭 (완전일치 → 포함매칭 → 토큰매칭)
4. 번지 정보 추출 및 API 파라미터 생성
5. 건축HUB API 호출 및 기본개요 데이터 수집
6. 결과를 CSV 파일로 저장


In [23]:
# 기본개요 수집 결과 데이터 로드 및 MGM_BLD_PK 변수 생성
def load_and_process_result():
    """기본개요 수집 결과를 로드하고 MGM_BLD_PK 변수를 생성"""
    
    # 결과 파일 로드
    result_file = "../result/기본개요_API요청시트_20250612_113614.csv"
    
    try:
        df_result = pd.read_csv(result_file, encoding='utf-8-sig')
        print(f"✓ 결과 파일 로드 성공: {result_file}")
        print(f"  데이터 크기: {df_result.shape}")
        print(f"  컬럼: {list(df_result.columns)}")
        
        # mgmBldrgstPk와 sigunguCd 컬럼 확인
        if 'mgmBldrgstPk' not in df_result.columns:
            print("❌ mgmBldrgstPk 컬럼이 없습니다.")
            return None
            
        if 'sigunguCd' not in df_result.columns:
            print("❌ sigunguCd 컬럼이 없습니다.")
            return None
        
        print(f"\n=== 기존 데이터 확인 ===")
        print(f"mgmBldrgstPk 결측치: {df_result['mgmBldrgstPk'].isna().sum()}개")
        print(f"sigunguCd 결측치: {df_result['sigunguCd'].isna().sum()}개")
        
        # mgmBldrgstPk 길이 분포 확인
        df_result['mgm_length'] = df_result['mgmBldrgstPk'].astype(str).str.len()
        print(f"\nmgmBldrgstPk 길이 분포:")
        print(df_result['mgm_length'].value_counts().sort_index())
        
        return df_result
        
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {result_file}")
        return None
    except Exception as e:
        print(f"❌ 파일 로드 오류: {e}")
        return None

# 데이터 로드
df_result = load_and_process_result()


✓ 결과 파일 로드 성공: ../result/기본개요_API요청시트_20250612_113614.csv
  데이터 크기: (449, 36)
  컬럼: ['rnum', 'platPlc', 'sigunguCd', 'bjdongCd', 'platGbCd', 'bun', 'ji', 'mgmBldrgstPk', 'mgmUpBldrgstPk', 'bldgId', 'regstrGbCd', 'regstrGbCdNm', 'regstrKindCd', 'regstrKindCdNm', 'newPlatPlc', 'bldNm', 'splotNm', 'block', 'lot', 'bylotCnt', 'naRoadCd', 'naBjdongCd', 'naUgrndCd', 'naMainBun', 'naSubBun', 'jiyukCd', 'jiguCd', 'guyukCd', 'jiyukCdNm', 'jiguCdNm', 'guyukCdNm', 'crtnDay', 'original_index', 'original_address', 'match_method', 'matched_address']

=== 기존 데이터 확인 ===
mgmBldrgstPk 결측치: 0개
sigunguCd 결측치: 0개

mgmBldrgstPk 길이 분포:
mgm_length
8      20
9     279
10     11
11     53
14     81
22      5
Name: count, dtype: int64


In [24]:
# MGM_BLD_PK 변수 생성 함수
def create_mgm_bld_pk(row):
    """
    mgmBldrgstPk와 sigunguCd를 이용해 MGM_BLD_PK 생성
    
    규칙:
    - mgmBldrgstPk가 21글자 이하: sigunguCd(5글자)-mgmBldrgstPk에서 앞 5글자 제거
    - mgmBldrgstPk가 22글자 이상: sigunguCd(5글자)-mgmBldrgstPk 그대로
    
    예시: 113711814 → 51190-1814 (sigunguCd가 51190일 때)
    """
    mgm = str(row['mgmBldrgstPk'])
    sigungu = str(row['sigunguCd'])
    
    # 결측치 처리
    if mgm == 'nan' or sigungu == 'nan':
        return None
    
    # mgmBldrgstPk 길이에 따른 처리
    if len(mgm) <= 21:
        # 21글자 이하: 앞 5글자 제거
        if len(mgm) >= 5:
            mgm_processed = mgm[5:]  # 앞 5글자 제거
        else:
            mgm_processed = mgm  # 5글자 미만이면 그대로
        result = f"{sigungu}-{mgm_processed}"
    else:
        # 22글자 이상: 그대로 사용
        result = f"{sigungu}-{mgm}"
    
    return result

# MGM_BLD_PK 변수 생성 및 적용
if df_result is not None:
    print("=== MGM_BLD_PK 변수 생성 ===")
    
    # MGM_BLD_PK 생성
    df_result['MGM_BLD_PK'] = df_result.apply(create_mgm_bld_pk, axis=1)
    
    # 결과 확인
    print(f"MGM_BLD_PK 생성 완료!")
    print(f"MGM_BLD_PK 결측치: {df_result['MGM_BLD_PK'].isna().sum()}개")
    
    # 샘플 확인 (길이별로)
    print(f"\n=== 변환 결과 샘플 ===")
    
    # 21글자 이하 샘플
    short_samples = df_result[df_result['mgm_length'] <= 21].head(5)
    if len(short_samples) > 0:
        print(f"\n📋 21글자 이하 변환 결과 (앞 5글자 제거):")
        for idx, row in short_samples.iterrows():
            print(f"  원본: {row['mgmBldrgstPk']} (길이: {row['mgm_length']})")
            print(f"  변환: {row['MGM_BLD_PK']}")
            print(f"  시군구: {row['sigunguCd']}")
            print()
    
    # 22글자 이상 샘플
    long_samples = df_result[df_result['mgm_length'] >= 22].head(3)
    if len(long_samples) > 0:
        print(f"📋 22글자 이상 변환 결과 (그대로 사용):")
        for idx, row in long_samples.iterrows():
            print(f"  원본: {row['mgmBldrgstPk']} (길이: {row['mgm_length']})")
            print(f"  변환: {row['MGM_BLD_PK']}")
            print(f"  시군구: {row['sigunguCd']}")
            print()
    
    # 길이별 통계
    print(f"=== 길이별 변환 통계 ===")
    length_stats = df_result.groupby('mgm_length').size()
    print(f"21글자 이하: {length_stats[length_stats.index <= 21].sum()}건")
    print(f"22글자 이상: {length_stats[length_stats.index >= 22].sum()}건")
    
    # 최종 데이터 확인
    print(f"\n=== 최종 데이터 확인 ===")
    print(f"전체 데이터: {len(df_result)}건")
    print(f"MGM_BLD_PK 생성 완료: {df_result['MGM_BLD_PK'].notna().sum()}건")
    print(f"MGM_BLD_PK 결측치: {df_result['MGM_BLD_PK'].isna().sum()}건")

else:
    print("데이터가 로드되지 않아서 MGM_BLD_PK를 생성할 수 없습니다.")


=== MGM_BLD_PK 변수 생성 ===
MGM_BLD_PK 생성 완료!
MGM_BLD_PK 결측치: 0개

=== 변환 결과 샘플 ===

📋 21글자 이하 변환 결과 (앞 5글자 제거):
  원본: 113711814 (길이: 9)
  변환: 51190-1814
  시군구: 51190

  원본: 113711815 (길이: 9)
  변환: 51190-1815
  시군구: 51190

  원본: 11371100174720 (길이: 14)
  변환: 51190-100174720
  시군구: 51190

  원본: 11371100174721 (길이: 14)
  변환: 51190-100174721
  시군구: 51190

  원본: 10971152859 (길이: 11)
  변환: 41194-152859
  시군구: 41194

📋 22글자 이상 변환 결과 (그대로 사용):
  원본: 1000000000000005146258 (길이: 22)
  변환: 52113-1000000000000005146258
  시군구: 52113

  원본: 1000000000000003970353 (길이: 22)
  변환: 44150-1000000000000003970353
  시군구: 44150

  원본: 1000000000000003970353 (길이: 22)
  변환: 44150-1000000000000003970353
  시군구: 44150

=== 길이별 변환 통계 ===
21글자 이하: 444건
22글자 이상: 5건

=== 최종 데이터 확인 ===
전체 데이터: 449건
MGM_BLD_PK 생성 완료: 449건
MGM_BLD_PK 결측치: 0건


In [25]:
# 처리된 데이터 저장
def save_processed_data():
    """MGM_BLD_PK가 추가된 데이터를 저장"""
    
    if df_result is None:
        print("❌ 저장할 데이터가 없습니다.")
        return
    
    # 현재 시각을 파일명에 포함하고 result 폴더에 저장
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"../result/기본개요_MGM_BLD_PK_추가_{timestamp}.csv"
    
    # result 폴더가 없으면 생성
    os.makedirs("../result", exist_ok=True)
    
    try:
        # 불필요한 임시 컬럼 제거
        df_save = df_result.drop(columns=['mgm_length'], errors='ignore')
        
        # CSV 저장
        df_save.to_csv(filename, index=False, encoding="utf-8-sig")
        
        print(f"✅ MGM_BLD_PK 추가된 데이터 저장 완료!")
        print(f"💾 저장 파일: {filename}")
        print(f"📊 저장된 데이터 크기: {df_save.shape}")
        
        # 주요 컬럼 확인
        key_columns = ['sigunguCd', 'mgmBldrgstPk', 'MGM_BLD_PK', 'platPlc', 'original_address']
        available_columns = [col for col in key_columns if col in df_save.columns]
        
        if available_columns:
            print(f"\n📋 저장된 데이터 미리보기 (주요 컬럼):")
            print(df_save[available_columns].head())
        
        return filename
        
    except Exception as e:
        print(f"❌ 데이터 저장 오류: {e}")
        return None

# 데이터 저장 실행
if df_result is not None:
    saved_file = save_processed_data()
    
    print(f"\n" + "="*60)
    print("🎉 MGM_BLD_PK 변수 생성 및 저장 완료!")
    print("="*60)
    print(f"📝 처리 결과:")
    print(f"  - 원본 데이터: ../result/기본개요_API요청시트_20250612_113614.csv")
    print(f"  - 처리된 데이터: {saved_file}")
    print(f"  - 추가된 변수: MGM_BLD_PK")
    print(f"  - 변환 규칙:")
    print(f"    • 21글자 이하: sigunguCd-mgmBldrgstPk(앞5글자제거)")
    print(f"    • 22글자 이상: sigunguCd-mgmBldrgstPk(그대로)")
else:
    print("데이터 로드에 실패해서 저장할 수 없습니다.")


✅ MGM_BLD_PK 추가된 데이터 저장 완료!
💾 저장 파일: ../result/기본개요_MGM_BLD_PK_추가_20250612_131507.csv
📊 저장된 데이터 크기: (449, 37)

📋 저장된 데이터 미리보기 (주요 컬럼):
   sigunguCd    mgmBldrgstPk       MGM_BLD_PK                   platPlc  \
0      51190       113711814       51190-1814  강원특별자치도 태백시 황지동 49-225번지   
1      51190       113711815       51190-1815  강원특별자치도 태백시 황지동 49-225번지   
2      51190  11371100174720  51190-100174720  강원특별자치도 태백시 황지동 49-225번지   
3      51190  11371100174721  51190-100174721  강원특별자치도 태백시 황지동 49-225번지   
4      41194     10971152859     41194-152859     경기도 부천시 소사구 소사본동 51번지   

              original_address  
0       강원특별자치도 태백시 황지동 49-225  
1       강원특별자치도 태백시 황지동 49-225  
2       강원특별자치도 태백시 황지동 49-225  
3       강원특별자치도 태백시 황지동 49-225  
4  경기도 부천시 소사구 소사본동 51번지, 소사역사  

🎉 MGM_BLD_PK 변수 생성 및 저장 완료!
📝 처리 결과:
  - 원본 데이터: ../result/기본개요_API요청시트_20250612_113614.csv
  - 처리된 데이터: ../result/기본개요_MGM_BLD_PK_추가_20250612_131507.csv
  - 추가된 변수: MGM_BLD_PK
  - 변환 규칙:
    • 21글자 이하: sigunguCd-mg

In [14]:
collect_basic_outline_data()


=== 전체 데이터 주소 파싱 시작 ===
처리할 데이터: 148건


주소 파싱: 100%|██████████| 148/148 [01:09<00:00,  2.14it/s]



=== 주소 파싱 결과 ===
✓ API 호출 가능: 100건
⚠️ 매핑 실패: 45건
❌ 파싱 실패: 3건

매칭 방법별 분포:
  스마트매칭(포함매칭(0.90)): 31건
  스마트매칭(토큰매칭(0.75)): 16건
  스마트매칭(토큰매칭(0.60)): 16건
  스마트매칭(토큰매칭(0.80)): 10건
  스마트매칭(포함매칭(0.92)): 8건
  스마트매칭(포함매칭(0.89)): 6건
  스마트매칭(포함매칭(0.85)): 5건
  스마트매칭(토큰매칭(0.67)): 4건
  스마트매칭(포함매칭(0.38)): 1건
  스마트매칭(포함매칭(0.56)): 1건
  스마트매칭(포함매칭(0.44)): 1건
  스마트매칭(포함매칭(0.51)): 1건

=== 기본개요 데이터 수집 시작 ===


API 호출 중: 100%|██████████| 100/100 [00:55<00:00,  1.79it/s]



=== 수집 완료 ===
✓ 총 449건 수집됨
❌ 실패: 49건
💾 저장 파일: ../result/기본개요_API요청시트_20250612_113614.csv

=== 결과 미리보기 ===
                    platPlc             original_address       match_method
0  강원특별자치도 태백시 황지동 49-225번지       강원특별자치도 태백시 황지동 49-225  스마트매칭(토큰매칭(0.75))
1  강원특별자치도 태백시 황지동 49-225번지       강원특별자치도 태백시 황지동 49-225  스마트매칭(토큰매칭(0.75))
2  강원특별자치도 태백시 황지동 49-225번지       강원특별자치도 태백시 황지동 49-225  스마트매칭(토큰매칭(0.75))
3  강원특별자치도 태백시 황지동 49-225번지       강원특별자치도 태백시 황지동 49-225  스마트매칭(토큰매칭(0.75))
4     경기도 부천시 소사구 소사본동 51번지  경기도 부천시 소사구 소사본동 51번지, 소사역사  스마트매칭(토큰매칭(0.67))


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,mgmUpBldrgstPk,bldgId,...,jiguCd,guyukCd,jiyukCdNm,jiguCdNm,guyukCdNm,crtnDay,original_index,original_address,match_method,matched_address
0,1,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,0049,0225,113711814,11371100174721,2120042350000779,...,,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
1,2,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,0049,0225,113711815,11371100174721,2120072350000101,...,,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
2,3,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,0049,0225,11371100174720,11371100174721,2120092350000176,...,,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
3,4,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,0049,0225,11371100174721,0,2020092350000029,...,,,자연녹지지역,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
4,1,경기도 부천시 소사구 소사본동 51번지,41194,10100,0,0051,0000,10971152859,10971152406,2120041980000004,...,,,,,,20220909,6,"경기도 부천시 소사구 소사본동 51번지, 소사역사",스마트매칭(토큰매칭(0.67)),경기도 부천시 소사구 소사본동 64번지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,10,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,0175,0001,11591100221977,1159112682,2220212560000135,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지
445,11,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,0175,0001,11591100226666,0,2020222560000025,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지
446,12,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,0175,0001,11591100221974,1159112682,2220212560000132,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지
447,13,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,0175,0001,11591100221979,1159112682,2220212560000137,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지


In [21]:
df_result = pd.read_csv("../result/기본개요_API요청시트_20250612_113614.csv")
df_result

,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,mgmUpBldrgstPk,bldgId,...,jiguCd,guyukCd,jiyukCdNm,jiguCdNm,guyukCdNm,crtnDay,original_index,original_address,match_method,matched_address
0,1,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,113711814,11371100174721,2120042350000779,...,,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
1,2,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,113711815,11371100174721,2120072350000101,...,,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
2,3,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,11371100174720,11371100174721,2120092350000176,...,,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
3,4,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,11371100174721,0,2020092350000029,...,,,자연녹지지역,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지
4,1,경기도 부천시 소사구 소사본동 51번지,41194,10100,0,51,0,10971152859,10971152406,2120041980000004,...,,,,,,20220909,6,"경기도 부천시 소사구 소사본동 51번지, 소사역사",스마트매칭(토큰매칭(0.67)),경기도 부천시 소사구 소사본동 64번지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,10,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100221977,1159112682,2220212560000135,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지
445,11,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100226666,0,2020222560000025,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지
446,12,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100221974,1159112682,2220212560000132,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지
447,13,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100221979,1159112682,2220212560000137,...,,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지


In [28]:
df_result = pd.read_csv("../result/기본개요_MGM_BLD_PK_추가_20250612_131507.csv")
df_result


,rnum,platPlc,sigunguCd,bjdongCd,platGbCd,bun,ji,mgmBldrgstPk,mgmUpBldrgstPk,bldgId,...,guyukCd,jiyukCdNm,jiguCdNm,guyukCdNm,crtnDay,original_index,original_address,match_method,matched_address,MGM_BLD_PK
0,1,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,113711814,11371100174721,2120042350000779,...,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지,51190-1814
1,2,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,113711815,11371100174721,2120072350000101,...,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지,51190-1815
2,3,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,11371100174720,11371100174721,2120092350000176,...,,,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지,51190-100174720
3,4,강원특별자치도 태백시 황지동 49-225번지,51190,10100,0,49,225,11371100174721,0,2020092350000029,...,,자연녹지지역,,,20221112,1,강원특별자치도 태백시 황지동 49-225,스마트매칭(토큰매칭(0.75)),강원특별자치도 태백시 황지동 20-8번지,51190-100174721
4,1,경기도 부천시 소사구 소사본동 51번지,41194,10100,0,51,0,10971152859,10971152406,2120041980000004,...,,,,,20220909,6,"경기도 부천시 소사구 소사본동 51번지, 소사역사",스마트매칭(토큰매칭(0.67)),경기도 부천시 소사구 소사본동 64번지,41194-152859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,10,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100221977,1159112682,2220212560000135,...,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지,43720-100221977
445,11,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100226666,0,2020222560000025,...,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지,43720-100226666
446,12,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100221974,1159112682,2220212560000132,...,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지,43720-100221974
447,13,충청북도 보은군 보은읍 이평리 175-1번지,43720,25031,0,175,1,11591100221979,1159112682,2220212560000137,...,,,,,20220924,145,충청북도 보은군 보은읍 이평리 175-1,스마트매칭(토큰매칭(0.80)),충청북도 보은군 보은읍 이평리 45번지,43720-100221979


In [30]:
# MGM_UP_PK 변수 생성 함수
def create_mgm_up_pk(row):
    """
    mgmUpBldrgstPK와 sigunguCd를 이용해 MGM_UP_PK 생성
    
    규칙:
    - mgmUpBldrgstPK가 21글자 이하: sigunguCd(5글자)-mgmUpBldrgstPK에서 앞 5글자 제거
    - mgmUpBldrgstPK가 22글자 이상: sigunguCd(5글자)-mgmUpBldrgstPK 그대로
    """
    mgm_up = str(row['mgmUpBldrgstPk'])
    sigungu = str(row['sigunguCd'])
    
    # 결측치 처리
    if mgm_up == 'nan' or sigungu == 'nan':
        return None
    
    # mgmUpBldrgstPK 길이에 따른 처리
    if len(mgm_up) <= 21:
        # 21글자 이하: 앞 5글자 제거
        if len(mgm_up) >= 5:
            mgm_processed = mgm_up[5:]  # 앞 5글자 제거
        else:
            mgm_processed = mgm_up  # 5글자 미만이면 그대로
        result = f"{sigungu}-{mgm_processed}"
    else:
        # 22글자 이상: 그대로 사용
        result = f"{sigungu}-{mgm_up}"
    
    return result

# MGM_UP_PK 변수 생성
df_result['MGM_UP_PK'] = df_result.apply(create_mgm_up_pk, axis=1)

print(f"MGM_UP_PK 생성 완료!")
print(f"MGM_UP_PK 결측치: {df_result['MGM_UP_PK'].isna().sum()}개")

MGM_UP_PK 생성 완료!
MGM_UP_PK 결측치: 0개


In [31]:
df_result['MGM_UP_PK'].value_counts()

MGM_UP_PK
11710-347          133
26410-382           84
44150-642           42
44150-489           40
52113-100185689     29
52113-118577        18
52113-0             13
11710-110           12
11710-0              9
43720-12682          8
44150-0              7
41650-3886           7
11620-0              5
26410-0              4
26260-1257           4
51190-100174721      3
43720-100226666      3
47230-1247           3
11620-100215785      3
52730-100182830      3
47230-0              2
46890-0              2
41194-152406         2
26260-0              2
43720-0              2
51190-0              1
11530-0              1
41194-0              1
41830-0              1
41650-0              1
52113-100203475      1
52730-0              1
50110-0              1
43730-0              1
Name: count, dtype: int64

In [ ]:
df_result

In [22]:
df = pd.read_excel("../data/batch_stage_matching_result_final_추가작업 요청.xlsx", sheet_name="총괄표제부 작업 필요")
df = df[df['Unnamed: 26'].isna()]
df

,SEQ_NO,RECAP_PK,연면적,사용승인연도,기관명,건축물명,new_주소,지상,지하,TOTAREA,...,EBD_COUNT,BD_COUNT,EBD_OVER_BD,건축물 등록일,대학일괄_matching_step,대학일괄_tokens,대학일괄_matched_date,대학일괄_matched_area,대학일괄_matched_tokens,Unnamed: 26
9,5793677,51770-12288,6515.750,1999-11-10,한국가스공사,한국가스공사정선연수원,"강원특별자치도 정선군 북평면 남평리 66-3, 한국가스공사생활관가동,경비실",5.0,NaN,NaN,...,NaN,NaN,NaN,2019-08-21,NaN,{'한국가스공사정선연수원'},NaN,NaN,NaN,NaN
12,331060,51190-100174721,6833.000,2003-06-30,강원특별자치도 태백시,태백시문화예술회관,강원특별자치도 태백시 황지동 49-225,3.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,{'태백시문화예술회관'},NaN,NaN,NaN,NaN
13,6371995,51760-893,8808.160,2001-04-02,한국청소년활동진흥원,평창수련원 제 가동 연수동,강원특별자치도 평창군 용평면 백옥포리 251,4.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,"{'평창수련원', '제', '가동', '연수동'}",NaN,NaN,NaN,NaN
14,6371996,51760-893,7436.870,2001-04-02,한국청소년활동진흥원,평창수련원 제 나동 숙박동,강원특별자치도 평창군 용평면 백옥포리 251,4.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,"{'숙박동', '나동', '평창수련원', '제'}",NaN,NaN,NaN,NaN
19,3271282,41281-1265,3980.000,1987-05-19,서울교통공사,지축차량기지 3동(복리후생관),경기도 고양시 덕양구 지축동 737-1 지축차량기지,3.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,"{'지축차량기지', '3동', '복리후생관'}",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,1631700,NaN,4073.980,1991-09-13,공주대학교,5동(한얼배움터),충청남도 공주시 옥룡동 201,4.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,"{'5동', '한얼배움터'}",NaN,NaN,NaN,NaN
380,1631701,NaN,3852.406,1999-10-11,공주대학교,9동(자료보존관),충청남도 공주시 옥룡동 201,4.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,"{'9동', '정보관'}",NaN,NaN,NaN,NaN
392,557806,NaN,4589.870,1994-08-18,충청북도 보은군,보은문화예술회관,충청북도 보은군 보은읍 이평리 175-1,3.0,1.0,NaN,...,NaN,NaN,NaN,2017-04-05,NaN,{'보은문화예술회관'},NaN,NaN,NaN,NaN
394,5362569,NaN,6204.480,2003-12-06,충북도립대학,학생생활관,충청북도 옥천군 옥천읍 금구리 85-20,5.0,1.0,NaN,...,6.0,14.0,no,2017-04-05,NaN,{'학생생활관'},NaN,NaN,NaN,NaN


In [32]:
# df_result와 df를 MGM_BLD_PK로 매칭하여 RECAP_PK 채우기
def fill_recap_pk():
    """
    df_result의 MGM_UP_PK를 이용해 df의 RECAP_PK를 채우는 함수
    """
    
    # 매칭을 위한 딕셔너리 생성 (MGM_BLD_PK -> MGM_UP_PK)
    mgm_mapping = {}
    
    for idx, row in df_result.iterrows():
        mgm_bld_pk = row.get('MGM_BLD_PK')
        mgm_up_pk = row.get('MGM_UP_PK')
        
        if pd.notna(mgm_bld_pk) and pd.notna(mgm_up_pk):
            mgm_mapping[mgm_bld_pk] = mgm_up_pk
    
    print(f"매핑 가능한 MGM_BLD_PK: {len(mgm_mapping)}개")
    
    # df의 RECAP_PK 채우기
    matched_count = 0
    not_found_count = 0
    
    for idx, row in df.iterrows():
        mgm_bld_pk = row.get('MGM_BLD_PK')
        
        if pd.notna(mgm_bld_pk):
            if mgm_bld_pk in mgm_mapping:
                # 매칭되는 경우: MGM_UP_PK 값으로 채우기
                df.loc[idx, 'RECAP_PK'] = mgm_mapping[mgm_bld_pk]
                matched_count += 1
            else:
                # 매칭되지 않는 경우: '조회예정'으로 채우기
                df.loc[idx, 'RECAP_PK'] = '조회예정'
                not_found_count += 1
    
    print(f"\n=== RECAP_PK 채우기 결과 ===")
    print(f"✓ 매칭 성공: {matched_count}건")
    print(f"⚠️ 조회예정: {not_found_count}건")
    print(f"📊 RECAP_PK 결측치: {df['RECAP_PK'].isna().sum()}개")
    
    return matched_count, not_found_count

# RECAP_PK 채우기 실행
matched, not_found = fill_recap_pk()

# 결과 확인
print(f"\n=== 최종 결과 확인 ===")
print(f"RECAP_PK 값 분포:")
print(df['RECAP_PK'].value_counts(dropna=False))

# 샘플 확인
print(f"\n매칭된 데이터 샘플:")
matched_samples = df[df['RECAP_PK'].notna() & (df['RECAP_PK'] != '조회예정')].head(3)
if len(matched_samples) > 0:
    for idx, row in matched_samples.iterrows():
        print(f"  MGM_BLD_PK: {row['MGM_BLD_PK']} → RECAP_PK: {row['RECAP_PK']}")

print(f"\n조회예정 데이터 샘플:")
pending_samples = df[df['RECAP_PK'] == '조회예정'].head(3)
if len(pending_samples) > 0:
    for idx, row in pending_samples.iterrows():
        print(f"  MGM_BLD_PK: {row['MGM_BLD_PK']} → RECAP_PK: {row['RECAP_PK']}")

매핑 가능한 MGM_BLD_PK: 167개

=== RECAP_PK 채우기 결과 ===
✓ 매칭 성공: 40건
⚠️ 조회예정: 107건
📊 RECAP_PK 결측치: 0개

=== 최종 결과 확인 ===
RECAP_PK 값 분포:
RECAP_PK
조회예정               107
11710-347            9
26410-382            5
44150-642            4
44150-489            4
52113-118577         3
26260-1257           2
51190-100174721      1
41830-0              1
41194-152406         1
47230-0              1
41650-3886           1
11530-0              1
11620-100215785      1
47230-1247           1
46890-0              1
50110-0              1
52730-100182830      1
43720-0              1
43730-0              1
Name: count, dtype: int64

매칭된 데이터 샘플:
  MGM_BLD_PK: 51190-1814 → RECAP_PK: 51190-100174721
  MGM_BLD_PK: 41194-152859 → RECAP_PK: 41194-152406
  MGM_BLD_PK: 41830-100198105 → RECAP_PK: 41830-0

조회예정 데이터 샘플:
  MGM_BLD_PK: 51770-12288 → RECAP_PK: 조회예정
  MGM_BLD_PK: 51760-16891 → RECAP_PK: 조회예정
  MGM_BLD_PK: 51760-16905 → RECAP_PK: 조회예정


In [35]:
# 엑셀 파일로 저장
df.to_excel('../result/건축HUB_API_기본개요수집_결과.xlsx', index=False)
print("엑셀 파일이 저장되었습니다: 건축HUB_API_기본개요수집_결과.xlsx")

엑셀 파일이 저장되었습니다: 건축HUB_API_기본개요수집_결과.xlsx
